In [79]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import START, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langchain_core.messages import SystemMessage, trim_messages
from typing import Sequence

import bs4
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore


## Simple ChatBot

In [3]:
llm = ChatOllama(
    temperature=0.2,
    model='gemma3:1b'
)

# -- create prompt template
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please response to the user queries"),
        ("user","Question:{question}")
    ]
)

# -- create output parser
output_parser=StrOutputParser()

# -- create chain
chain=prompt|llm|output_parser

In [4]:
input_text = "hi, how are you?"
response = chain.invoke({'question':input_text})
print(response)

Hi there! I’m doing well, thanks for asking. As a large language model, I don’t really *feel* things, but I’m functioning perfectly and ready to help you with whatever you need. 😊 How about you?


## Advance ChatBot

### Without RAG

In [ ]:
# load llm
llm = ChatOllama(
    temperature=0.2,
    model='gemma3:1b'
)

# create prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Please response to the user queries."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# create trimmer
trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [ ]:
# define dictionary for state  
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
        
# create node function
def call_model(state: State):
    # lakukan trimmer pada history message
    trimmed_messages = trimmer.invoke(state["messages"])
    
    # masukan trimmed message dan language ke prompt template 
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages}
    )

    # generate jawaban dengan model LLM
    response = llm.invoke(prompt)

    return {"messages": [response]}

# buat workflow
workflow = StateGraph(state_schema=State)
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# -- compile workflow dengan memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



In [ ]:
# -- set config for memory state
# tiap state pada memory akan dibedakan berdasarkan thread_id nya
config = {"configurable": {"thread_id": "abc678"}}

output = app.invoke({
    "messages": [HumanMessage("hi my name is Bob")], 
}, config)

output = app.invoke({
    "messages": [HumanMessage("what is my name?")], 
}, config)

output = app.invoke({
    "messages": [HumanMessage("what your name?")], 
}, config)

output = app.invoke({
    "messages": [HumanMessage("what python? in 1 paragraph")], 
}, config)

output = app.invoke({
    "messages": [HumanMessage("what html? in 1 paragraph")], 
}, config)


In [7]:
for chat in output["messages"]:
    chat.pretty_print()

================================ Human Message =================================

hi my name is Bob
================================== Ai Message ==================================

Hi Bob! It’s nice to meet you. How can I help you today? 😊
================================ Human Message =================================

what is my name?
================================== Ai Message ==================================

Your name is Bob! 😊 

Is there anything you’d like to do with that information?
================================ Human Message =================================

what your name?
================================== Ai Message ==================================

As a large language model, I don't have a name in the way a person does. I was created by Google! 

But you could say… **Google AI**! 😄
================================ Human Message =================================

what python? in 1 paragraph
================================== Ai Message ==========================

### With RAG

In [66]:
# -- load document
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )
loader = PyPDFLoader('../resources/ISP Company FAQ.pdf')
docs = loader.load()

# -- split docs
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

# -- load to vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(), # vector db hanya akan disimpan di dalam memory selama runtime
    index_to_docstore_id={},
)
document_ids = vector_store.add_documents(documents=all_splits)

# -- Create retriever
retriever = vector_store.as_retriever()

In [80]:
# load llm
llm = ChatOllama(
    temperature=0.2,
    model='gemma3:1b'
)

# create prompt template with document retrieval
prompt_template_retrieval = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
                You are an assistant for question-answering tasks. 
                Use the following pieces of retrieved context to answer the question. 
                If you don't know the answer, say that you don't know. 
                Use three sentences maximum and keep the answer concise.
                
                retrieved context:
                {context}
            """
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# create trimmer
trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [36]:
# define dictionary for state  
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # language: str
        
# create node function
def call_model(state: State):
    # lakukan trimmer pada history message
    trimmed_messages = trimmer.invoke(state["messages"])

    last_message = state["messages"][-1]
    context = retriever.invoke(last_message.content)
    
    # masukan trimmed message dan language ke prompt template 
    prompt = prompt_template_retrieval.invoke({
        "context": context,
        "messages": trimmed_messages,
    })

    # generate jawaban dengan model LLM
    response = llm.invoke(prompt)

    return {"messages": [response]}

# buat workflow
workflow = StateGraph(state_schema=State)
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# -- compile workflow dengan memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



In [39]:
config = {"configurable": {"thread_id": "abc678"}}

output = app.invoke({
    "messages": [HumanMessage("whats my name?")], 
}, config)

for chat in output["messages"]:
    chat.pretty_print()

================================ Human Message =================================

hi my name is Bob
================================== Ai Message ==================================

Hi Bob! How can I help you today?
================================ Human Message =================================

Can I cancel my order
================================== Ai Message ==================================

Yes, you can cancel your order. Orders can be canceled within 24 hours of placement. After that, cancellations are not guaranteed.
================================ Human Message =================================

whats my name?
================================== Ai Message ==================================

I understand you’re looking for information about your name. However, I’m designed to help with questions and provide assistance, and I don’t have access to personal information like names. 

Perhaps you were thinking of another system or platform?


## Class Based

In [ ]:
# define dictionary for state  
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    

class AdvanceChatBot:
    def __init__(self):
        self.vector_store = self.__init_vector_store()
        self.text_splitter = self.__init_text_splitter()
        self.llm = self.__init_llm_model()
        self.trimmer = self.__init_trimmer()
        self.prompt_template = self.__init_prompt_template()
        self.prompt_template_with_retrieval = self.__init_prompt_template_with_retrieval()

        self.is_use_rag = False
    
    def __init_vector_store(self):
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
        vector_store = FAISS(
            embedding_function=embeddings,
            index=index,
            docstore=InMemoryDocstore(), # vector db hanya akan disimpan di dalam memory selama runtime
            index_to_docstore_id={},
        )
        return vector_store
    
    def __init_text_splitter(self):
        return RecursiveCharacterTextSplitter(
            chunk_size=1000,  # chunk size (characters)
            chunk_overlap=200,  # chunk overlap (characters)
            add_start_index=True,  # track index in original document
        )

    def __init_llm_model(self):
        # load the LLM model
        return ChatOllama(
            temperature=0.5,
            model='gemma3:1b'
        )
    
    def __init_trimmer(self):
        # create trimmer
        return trim_messages(
            max_tokens=1000,
            strategy="last",
            token_counter=self.llm,
            include_system=True,
            allow_partial=False,
            start_on="human",
        )
    
    def __init_prompt_template(self):
        # create prompt template
        return ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a helpful assistant."
                ),
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
    
    def __init_prompt_template_with_retrieval(self):
        return ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                    You are an assistant for question-answering tasks. 
                    Use the following pieces of retrieved context to answer the question. 
                    If you don't know the answer, say that you don't know. 
                    Use three sentences maximum and keep the answer concise.
                    
                    retrieved context:
                    {context}
                """
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    
    def load_document(self, docs_path):
        # load document
        loader = PyPDFLoader(docs_path)
        docs = loader.load()

        # split docs
        all_splits = self.text_splitter.split_documents(docs)

        # Store splitted document into vector_store
        self.vector_store.add_documents(documents=all_splits)

        # -- Create retriever
        self.retriever = self.vector_store.as_retriever()

        # set flag
        self.is_use_rag = True
        
    def load_model(self):
        # buat workflow
        workflow = StateGraph(state_schema=State)

        if self.is_use_rag:
            workflow.add_node("model", self.__generate_rag)
        else:
            workflow.add_node("model", self.__generate)
        
        workflow.add_edge(START, "model")

        # compile workflow dengan memory
        memory = MemorySaver()
        app = workflow.compile(checkpointer=memory)
        return app

    def __generate(self, state: State):
        # lakukan trimmer pada history message
        trimmed_messages = self.trimmer.invoke(state["messages"])

        # masukan trimmed message dan language ke prompt template 
        prompt = self.prompt_template.invoke(
            {"messages": trimmed_messages}
        )

        # generate jawaban dengan model LLM
        response = self.llm.invoke(prompt)

        return {"messages": [response]}
    
    def __generate_rag(self, state: State):
        # lakukan trimmer pada history message
        trimmed_messages = self.trimmer.invoke(state["messages"])

        last_message = state["messages"][-1]
        context = self.retriever.invoke(last_message.content)
        
        # masukan trimmed message dan language ke prompt template 
        prompt = self.prompt_template_retrieval.invoke({
            "context": context,
            "messages": trimmed_messages,
        })

        # generate jawaban dengan model LLM
        response = self.llm.invoke(prompt)

        return {"messages": [response]}

In [76]:
model = AdvanceChatBot()

# load workflow without rag
# bot = model.load_model()

# load workflow with rag
model.load_document('../resources/ISP Company FAQ - Copy.pdf')
bot = model.load_model()

In [78]:
config = {"configurable": {"thread_id": "abc123"}}
result = bot.invoke({
    "messages": [HumanMessage("how to contact customer service?")], 
    # "messages": [HumanMessage("hi")], 
}, config)

for chat in result["messages"]:
    chat.pretty_print()

================================ Human Message =================================

reset password?
================================== Ai Message ==================================

You can reset your Wi-fi password via our mobile app, website, or by contacting customer service.
================================ Human Message =================================

how to contact customer service?
================================== Ai Message ==================================

You can contact customer service through the following methods:

*   **Live Chat:** Visit our website: [https://www.example.com/contact](https://www.example.com/contact)
*   **Call Center:** Call us at: [Insert Phone Number Here]
*   **Email:** Send an email to: [Insert Email Address Here]

Please note that the contact details might change, so it’s always a good idea to check our website for the most up-to-date information.
